## Data 불러오기

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# transfrom할 때 Tensor로 무조건 바꾸어 줘야함.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [9]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [10]:
60000 / 64

937.5

In [4]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [7]:
len(train_dataloader)

938

## Custom Dataset
```python
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
```
* dataloader를 생성할 때 한 번만 실행. 초기화 개념

```python
    def __len__(self):
```
* 데이터셋의 샘플 갯수를 반환

```python
    def __getitem__(self, idx):
```
* dataloader에서 데이터를 불러오는 부분. index기반으로 load.

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        
        self.img_labels = pd.read_csv(annotations_file, names=['file_name', 'label'])
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        #image 경로 불러옴
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        #image를 읽음, data를 불러옴
        image = read_image(img_path)
        #image에 해당하는 label값, target을 불러옴
        label = self.img_labels.iloc[idx, 1]
        
        #transform을 할 경우
        if self.transform:
            image = self.transform(image)
            
        
        #target data에 transform을 할 경우
        if self.target_transform:
            label = self.target_transform(label)
            
        #image(data)와 label(target)
        return image, label

## Model

In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [18]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        #super().__init__()는 무조건 해줘야함
        #nn.Module을 상속
        #https://daebaq27.tistory.com/60
        
        super().__init__()
        
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
model = NeuralNetwork().to(device)

In [29]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

## Train

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    #X : fashion image, y : target label
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #model(X) : X를 input으로 했을 때 predict한 값
        pred = model(X)
        #loss : predict한 값과 실제 label값의 차이
        loss = loss_fn(pred, y)

        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #loss 출력
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.319883  [   64/60000]
loss: 2.308114  [ 6464/60000]
loss: 2.287620  [12864/60000]
loss: 2.277103  [19264/60000]
loss: 2.259905  [25664/60000]
loss: 2.239175  [32064/60000]
loss: 2.238439  [38464/60000]
loss: 2.208407  [44864/60000]
loss: 2.198475  [51264/60000]
loss: 2.174596  [57664/60000]
Test Error: 
 Accuracy: 36.6%, Avg loss: 2.167862 

Epoch 2
-------------------------------
loss: 2.178892  [   64/60000]
loss: 2.174331  [ 6464/60000]
loss: 2.111604  [12864/60000]
loss: 2.122827  [19264/60000]
loss: 2.087337  [25664/60000]
loss: 2.030715  [32064/60000]
loss: 2.051017  [38464/60000]
loss: 1.976904  [44864/60000]
loss: 1.970588  [51264/60000]
loss: 1.919686  [57664/60000]
Test Error: 
 Accuracy: 53.5%, Avg loss: 1.911475 

Epoch 3
-------------------------------
loss: 1.944860  [   64/60000]
loss: 1.923946  [ 6464/60000]
loss: 1.798850  [12864/60000]
loss: 1.832588  [19264/60000]
loss: 1.749633  [25664/60000]
loss: 1.697824  [32064/600

In [26]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [27]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [28]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
